In [1]:
# !pip install h3
# !pip install haversine

In [2]:
import pandas as pd
import h3
from haversine import haversine

# Region View

## Region - region (distance)

**Entrada:**
Check-ins com **latitude e longitude**

**Saída:**
**Matriz de adjacência** entre as regiões n x n, no qual **n** representa o número de regiões e cada linha **i j** representa a distância da região i até a região j em metros.

In [3]:
checkins = pd.read_csv('checkins.csv')

In [4]:
def generate_h3_cell(row):
    lat = row['latitude']
    lon = row['longitude']
    resolution = 9

    h3_cell = h3.geo_to_h3(lat, lon, resolution)
    return h3_cell

In [5]:
checkins['h3_cell'] = (
    checkins
    .apply(generate_h3_cell, axis=1)
)

checkins['h3_cell'].nunique()

2181

Utilizando a resolução 9, temos 2181 células h3.

Fazer um gráfico mapeando os checkins no estado de NY e fazer outro gráfico mapeando as tesselações no estado de NY, para validarmos se estamos pegando um número ideal de resolução

In [6]:
def h3_centroid_distance(cell_h3_1, cell_h3_2):
    centroid_1 = h3.h3_to_geo(cell_h3_1)
    centroid_2 = h3.h3_to_geo(cell_h3_2)
    
    distance_meters = haversine(centroid_1, centroid_2) * 1000
    return distance_meters

### Criando um map para as regiões

In [7]:
h3_regions = checkins['h3_cell'].unique()
regions_map = []

i = 0
for h3_region in h3_regions:
    region_map = {'region_id': i, 'value': h3_region}
    i = i+1
    regions_map.append(region_map)

### Calcula distância do primeiro em relação a todos do mapeamento

In [8]:
reference_region = regions_map[0]['value']
distances = []

for region_map in regions_map:    
    region_id = region_map['region_id']
    region_h3 = region_map['value']
    
    distance = h3_centroid_distance(reference_region, region_h3)
    distances.append(distance)

### Faz para todos as regiões

In [9]:
adjacency_matrix = []

for region_map_i in regions_map:
    region_h3_i = region_map_i['value']
    distances_i_j = []

    for region_map_j in regions_map:    
        region_id_j = region_map_j['region_id']
        region_h3_j = region_map_j['value']
        
        distance = h3_centroid_distance(region_h3_i, region_h3_j)
        distances_i_j.append(distance)
    
    adjacency_matrix.append(distances_i_j)

## Region - POI (quantity)

**Entrada:**

Checkins com uma **região h3** associada à latitude e longitude 

**Saída:**
**Matriz de features **contendo em cada linha** a região h3** e em cada coluna a **quantidade de check-ins em um dado POI**.

In [28]:
feature_matrix_poi_region_qt = (
    pd.crosstab(
        index=checkins['h3_cell'], columns=checkins['category']
    )
)
feature_matrix_poi_region_qt.columns = (
    [f'{col}' for col in feature_matrix_poi_region_qt.columns]
)